In [2]:
from langchain.chat_models import ChatOpenAI

chat_model = ChatOpenAI()

chat_model.predict("안녕? 간단하게 답해줘")

c:\Anaconda3\envs\PTLLM\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
c:\Anaconda3\envs\PTLLM\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'안녕하세요! 무엇을 도와드릴까요?'

In [4]:
from langchain.schema import HumanMessage

messages = [HumanMessage(content="안녕? 간단하게 답해줘")]
chat_model.predict_messages(messages, temperature = 0.1)

c:\Anaconda3\envs\PTLLM\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `predict_messages` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


AIMessage(content='안녕하세요! 무엇을 도와드릴까요?', response_metadata={'finish_reason': 'stop', 'logprobs': None})

In [9]:
from langchain.prompts import PromptTemplate

my_template = """아래의 질문에 대해 간단하게 답하면서 신나세요.
질문: {question}"""

prompt = PromptTemplate.from_template(my_template)

chat_model.predict(prompt.format(question="넌 누구야"))

PromptTemplate(input_variables=['question'], template='아래의 질문에 대해 간단하게 답하면서 신나세요.\n질문: {question}')

In [18]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

template = """
You are a helpful assistant to help teenagers learn {output_language}.
Answer the question in <{output_language}> within 1~2 sentences.
YOU MUST USE <{output_language}> TO ANSWER THE QUESTION.
Question: """
human_template = "{text}"

system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# chat_prompt.format(output_language="한국어", text='밥 줘') # => System, Human 프롬프트 분간 없이 그냥 전체 문자열로 제공됨

# 메시지 리스트를 인풋으로 줄거니까 predict_messages() 사용
chat_model.predict_messages(chat_prompt.format_messages(output_language="프랑스어", text="밥 줘"))

'System: \nYou are a helpful assistant to help teenagers learn 한국어.\nAnswer the question in <한국어> within 1~2 sentences.\nYOU MUST USE <한국어> TO ANSWER THE QUESTION.\nQuestion: \nHuman: 밥 줘'

In [19]:
from langchain.schema import BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser):
    """LLM 아웃풋에 있는 ','을 분리해서 리턴하는 파서."""

    def parse(self, text: str):
        return text.strip().split(", ")
    
CommaSeparatedListOutputParser().parse("아기, 여우")

['아기', '여우']

# **최종 조합**

In [20]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
from langchain.schema import BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser):
    """LLM 아웃풋에 있는 ','을 분리해서 리턴하는 파서"""

    def parse(self, text: str):
        return text.strip().split(", ")

template = """
너는 5세 아이의 낱말놀이를 도와주는 AI야.
아이가 어떤 카테고리에 해당하는 개체들을 말해달라고 <질문>을 하면
해당 카테고리에 해당하는 단어들을 5개 나열해야 해.
이때 각 단어는 반드시 comma(,)로 분리해서 대답해주고, 이외의 말은 하지 마.

질문: """
human_template = "{text}"

system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

chain = LLMChain(
    llm=ChatOpenAI(),
    prompt=chat_prompt,
    output_parser=CommaSeparatedListOutputParser()
)
chain.run('유사인간')


c:\Anaconda3\envs\PTLLM\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


['인공지능', '안드로이드', '클론', '로봇', '사이보그,']